# Imports

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import plot_tree

# Importing and Splitting data

In [25]:
modelEncoder = LabelEncoder()
transmissionEncoder = LabelEncoder()
fuelTypeEncoder = LabelEncoder()

def dataset(brand):

    file = pd.read_csv(brand, quotechar='"', skipinitialspace=True)

    modelEncoder.fit(file["model"])
    file["model"] = modelEncoder.transform(file["model"])

    transmissionEncoder.fit(file["transmission"])
    file["transmission"] = transmissionEncoder.transform(file["transmission"])

    fuelTypeEncoder.fit(file["fuelType"])
    file["fuelType"] = fuelTypeEncoder.transform(file["fuelType"])

    file = file.head(1000)
    # X = file.drop(columns = ['price'])
    # Y = file.price
    # print(file)

    X = file.drop(['price'], axis = 1).to_numpy()
    # X = file[['year', 'mileage', 'tax', 'mpg', 'engineSize']].to_numpy(dtype = 'int')
    Y = file['price'].values.reshape(-1,1)

#     print(X)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 601)
    
    return  X_train, X_test, Y_train, Y_test

In [26]:
X_train, X_test, Y_train, Y_test = dataset("UKUsedCarDataSet/audi.csv") # Use Audi dataset as default for KNN analysis

# Using Scikit-Learn

In [13]:
myTree = DecisionTreeRegressor(random_state = 601, max_depth = 3)
myTree.fit(X_train, Y_train)
print("R Squared: ", myTree.score(X_test, Y_test))

R Squared:  0.6928749622462103


# Visualising Decsion Tree:

In [14]:
X1 = file.drop(['price'], axis = 1)

plt.figure(figsize=(20, 10), dpi = 200)
plot_tree(myTree, feature_names = X1.columns, filled=True);

NameError: name 'file' is not defined

# RMSE

In [15]:
def rmse(h, y):
  return sqrt(mean_squared_error(h, y))

# Node Class

In [16]:
# Node class to initialise instances of each 
class Node():
    
    def __init__(self, feature = None, limit = None, leftSide = None, rightSide = None, gain = None, leaf = None):
        
        self.feature = feature
        self.limit = limit
        self.leftSide = leftSide
        self.rightSide = rightSide
        self.gain = gain
        self.leaf = leaf 

# Decision Tree Regression Class

In [17]:
class DTRegressor():
    def __init__(self, minSamples, maxDepth):
        self.root = None
        self.minSamples = minSamples
        self.maxDepth = maxDepth
        
    def infoGain(self, parent, leftNode, rightNode):
        leftWeight = len(leftNode) / len(parent)
        rightWeight = len(rightNode) / len(parent)
        
        information_gain = np.var(parent) - (leftWeight * np.var(leftNode) + rightWeight * np.var(rightNode))

        return information_gain
        
    def splitTree(self, trainingSet, feature, limit):
        leftBranch = []
        rightBranch = []
        for i in trainingSet:
            if i[feature] <= limit:
                leftBranch.append(i)
            else:
                rightBranch.append(i)
        rightBranch = np.array(rightBranch)
        leftBranch = np.array(leftBranch)
        return leftBranch, rightBranch
        
    def bestSplit(self, trainingSet, X):
        bestSplitt = {} 
        biggestGain = -1
        for feature in range(X.shape[1]): 
#             featureValues = trainingSet[:, feature] #current feature selected
            featureValues = []
            for i in range(len(trainingSet)):
                featureValues.append(trainingSet[i, feature])
            thresholds = np.unique(featureValues)
            for j in thresholds: #j = threshold
                leftSide, rightSide = self.splitTree(trainingSet, feature, j) #splits node into 2 sub-trees
                if (len(leftSide) > 0 and len(rightSide) > 0 ):
                    parent = []
                    for i in range(len(trainingSet)):
                        parent.append(trainingSet[i, -1])

                    leftNode = []
                    for i in range(len(leftSide)):
                        leftNode.append(leftSide[i, -1])
                        
                    rightNode = []
                    for i in range(len(rightSide)):
                        rightNode.append(rightSide[i, -1])

                    currentGain = self.infoGain(parent, leftNode, rightNode) 
                    if currentGain > biggestGain:
                        
                        bestSplitt["feature"] = feature
                        bestSplitt["limit"] = j
                        bestSplitt["leftSide"] = leftSide
                        bestSplitt["rightSide"] = rightSide
                        bestSplitt["gain"] = currentGain
                        biggestGain = currentGain
                
        return bestSplitt
   
        
    def treeBuild(self, trainingSet, currentDepth = 0):
        
#       #Split training into features and labels
        X = trainingSet[:,:-1] # everything but the last value
        Y = []
        for i in range(len(trainingSet)):
            Y.append(trainingSet[i, -1])# only the last value
        
        #iterates until this condition is met
        if X.shape[0] >= self.minSamples and currentDepth <= self.maxDepth:
#             bestSplit = self.bestSplit(trainingSet, samplesNumb, featuresNumb)
            bestSplitNode = self.bestSplit(trainingSet, X)
            
            if bestSplitNode["gain"] > 0:
                leftTree = self.treeBuild(bestSplitNode["leftSide"], currentDepth + 1)
                rightTree = self.treeBuild(bestSplitNode["rightSide"], currentDepth + 1)
                node = Node(bestSplitNode["feature"], bestSplitNode["limit"], leftTree, rightTree, bestSplitNode["gain"])
                
                return node
        
        leafValue = np.mean(Y) #calculates mean of leaf nodes
        val = Node(leaf = leafValue)
        return val
    
    def predictionLoop(self, testRow, root):
        if root.leaf != None: #not empty
            return root.leaf
        
        featureVal = testRow[root.feature]
        if featureVal <= root.limit:
            return self.predictionLoop(testRow, root.leftSide)
        else:
            return self.predictionLoop(testRow, root.rightSide)
        
   
    def predict(self, xTest):
        predictions = []
        for row in xTest:
            predictions.append(self.predictionLoop(row, self.root)) 
        return predictions

        
        
    def fit(self, X, Y):
        trainingSet = np.concatenate((X, Y), axis=1) #Joins training data back together
        self.root = self.treeBuild(trainingSet)

# Training the algorithm

In [18]:
myTree = DTRegressor(3, 3)  
myTree.fit(X_train, Y_train) #trains the model using the training set

# Evaluation

In [9]:
def evaluation(train, test, yTrain, yTest):    
    y_pred = myTree.predict(X_test)
    error = rmse(yTest, y_pred) 
    print('The RMSE value is:', error)
    for i in range(len(y_pred)):
        print("\nOriginal value:", yTest[i], "vs the predicted value:", y_pred[i])
        print("The difference is:", yTest[i] - y_pred[i])
evaluation(X_train, X_test, Y_train, Y_test)

The RMSE value is: 5486.187742617547

Original value: [18498] vs the predicted value: 17614.6
The difference is: [883.4]

Original value: [22995] vs the predicted value: 17614.6
The difference is: [5380.4]

Original value: [12495] vs the predicted value: 34189.666666666664
The difference is: [-21694.66666667]

Original value: [14500] vs the predicted value: 13291.496503496504
The difference is: [1208.5034965]

Original value: [24995] vs the predicted value: 20879.25806451613
The difference is: [4115.74193548]

Original value: [10200] vs the predicted value: 17614.6
The difference is: [-7414.6]

Original value: [21995] vs the predicted value: 20879.25806451613
The difference is: [1115.74193548]

Original value: [16800] vs the predicted value: 17614.6
The difference is: [-814.6]

Original value: [12798] vs the predicted value: 13291.496503496504
The difference is: [-493.4965035]

Original value: [20498] vs the predicted value: 54140.307692307695
The difference is: [-33642.30769231]

Orig

# User Input Predictions

In [27]:
inputPred = []
entries = []

def userInput():
    chooseBrand = input("Choose your car brand: Audi, BMW, Ford, Hyundai, Mercedes, Skoda, Toyota, Vauxhall or Volkswagen \n")
    
    if chooseBrand == "Audi":
        return "UKUsedCarDataSet/audi.csv"
    elif chooseBrand == "BMW":
        return "UKUsedCarDataSet/bmw.csv"
    elif chooseBrand == "Ford":
        return "UKUsedCarDataSet/ford.csv"
    elif chooseBrand == "Hyundai":
        return "UKUsedCarDataSet/hyundi.csv"
    elif chooseBrand == "Mercedes":
        return "UKUsedCarDataSet/merc.csv"
    elif chooseBrand == "Skoda":
        return "UKUsedCarDataSet/skoda.csv"
    elif chooseBrand == "Toyota":
        return "UKUsedCarDataSet/toyota.csv"
    elif chooseBrand == "Vauxhall":
        return "UKUsedCarDataSet/vauxhall.csv"
    elif chooseBrand == "Volkswagen":
        return "UKUsedCarDataSet/vw.csv"
    else:
        print("Invalid Car Brand")
        userInput()
    return 

X_train, X_test, Y_train, Y_test = dataset(userInput())
print("\n ***Training Tree Model***")
myTree = DTRegressor(3, 3)  
myTree.fit(X_train, Y_train)

print("\n List of models:")
print(list(modelEncoder.classes_))
inputPred.append((modelEncoder.transform([input("\nWhat Model is your car? ")]))[0])
inputPred.append(int(input("What year is your car? ")))
inputPred.append((transmissionEncoder.transform([input("What transmission is your car? ")]))[0])
inputPred.append(int(input("How much mileage does your car have? ")))
inputPred.append((fuelTypeEncoder.transform([input("What's your car fuel type? ")]))[0])
inputPred.append(int(input("How much is your cars tax? ")))
inputPred.append(float(input("What's MPG of your car? ")))
inputPred.append(float(input("What the engine size of your car? ")))
entries.append(inputPred)
# inputPred = scaler.transform([inputPred])

import time
print("\n ***Predicting***")
start = time.time()
y_pred = myTree.predict([inputPred])
# {0:.2f}'.format()
print("\n Predicted price for your car is: £", y_pred[0])

print("\n ***Predicted in", time.time() - start,"seconds***")

# RS6,2016,Semi-Auto,49050,Petrol,325,29.4,4.0 -- Price is £44,985   Pred = £41,233.30
# BMW,5 Series,2019,Semi-Auto,4405,Petrol,145,48.7,2.0     Price = £26,000 Pred = £27,077.49 

Choose your car brand: Audi, BMW, Ford, Hyundi, Mercedes, Skoda, Toyota, Vauxhall or Volkswagen 
BMW

 ***Training Tree Model***

 List of models:
['1 Series', '2 Series', '3 Series', '4 Series', '5 Series', '6 Series', '7 Series', '8 Series', 'M2', 'M3', 'M4', 'M5', 'M6', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'Z3', 'Z4', 'i3', 'i8']

What Model is your car? 5 Series
What year is your car? 2019
What transmission is your car? Semi-Auto
How much mileage does your car have? 4405
What's your car fuel type? Petrol
How much is your cars tax? 145
What's MPG of your car? 48.7
What the engine size of your car? 2

 ***Predicting***

 Predicted price for your car is: £ 27077.494623655915

 ***Predicted in 0.0 seconds***
